In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
columns = [
    "Name", "platform", "Year_of_Release", "Genre",
    "Publisher", "NA_Salesr", "EU_Sales","JP_Sales",
    "Other_Sales","Global_Sales","Critic_Score","Critic_Count","User_Score","User_Count","Developer","Rating"
]

target = ["Global_Sales"]

In [5]:
# Load the data
file_path = Path('/Users/Sheem/OneDrive/Desktop/Analysis Projects/Module 17/vgsales_clean.csv')
df = pd.read_csv(file_path, skiprows=0)



# Drop the null rows
df = df.dropna()

df.head(10)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,The Sims 3,PC,2009.0,Simulation,Electronic Arts,0.99,6.42,0.00,0.60,8.01,86.0,75.0,7.6,886.0,The Sims Studio,T
1,World of Warcraft,PC,2004.0,Role-Playing,Activision,0.08,6.21,0.00,0.00,6.29,93.0,57.0,7.3,2182.0,Blizzard Entertainment,T
2,Diablo III,PC,2012.0,Role-Playing,Activision,2.44,2.16,0.00,0.54,5.14,88.0,86.0,4.0,9629.0,Blizzard Entertainment,M
4,StarCraft II: Wings of Liberty,PC,2010.0,Strategy,Activision,2.57,1.68,0.00,0.58,4.84,93.0,82.0,8.2,3052.0,Blizzard Entertainment,T
6,Half-Life,PC,1997.0,Shooter,Vivendi Games,4.03,0.00,0.09,0.00,4.12,96.0,24.0,9.1,3161.0,Valve Software,M
7,World of Warcraft: The Burning Crusade,PC,2007.0,Role-Playing,Activision,2.57,1.52,0.00,0.00,4.09,91.0,46.0,7.9,785.0,Blizzard Entertainment,T
8,The Elder Scrolls V: Skyrim,PC,2011.0,Role-Playing,Bethesda Softworks,1.16,2.12,0.00,0.60,3.88,94.0,32.0,8.1,9073.0,Bethesda Game Studios,M
9,The Sims: Unleashed,PC,2002.0,Simulation,Electronic Arts,2.03,1.56,0.00,0.17,3.76,79.0,21.0,8.0,29.0,Maxis,T
11,The Sims: Vacation,PC,2002.0,Simulation,Electronic Arts,1.72,1.21,0.00,0.14,3.07,75.0,18.0,7.8,22.0,Maxis,T
12,The Sims 4,PC,2014.0,Simulation,Electronic Arts,1.00,1.82,0.00,0.23,3.05,70.0,74.0,3.9,2057.0,Maxis,T


In [6]:
# Create our features
X = pd.get_dummies(df.drop('Global_Sales', axis=1))

# Create our target
y = df['Developer']

In [7]:
X.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Name_ Tales of Xillia 2,...,Developer_Zoe Mode,Developer_Zombie Studios,Developer_ZootFly,Developer_id Software,"Developer_id Software, Raven Software",Rating_E,Rating_E10+,Rating_M,Rating_RP,Rating_T
count,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,...,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000,1818.000000
mean,2010.793179,0.310847,0.292695,0.035352,0.100094,73.322332,32.514301,6.851100,447.143014,0.000550,...,0.001100,0.001100,0.000550,0.003300,0.001100,0.186469,0.125963,0.338284,0.000550,0.348735
std,3.700409,0.598879,0.643347,0.109943,0.221769,12.898382,20.088058,1.455917,1003.156611,0.023453,...,0.033159,0.033159,0.023453,0.057369,0.033159,0.389592,0.331898,0.473256,0.023453,0.476701
min,1985.000000,0.000000,0.000000,0.000000,0.000000,13.000000,4.000000,0.500000,4.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.000000,0.010000,0.020000,0.000000,0.010000,66.000000,17.000000,6.100000,34.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2011.000000,0.100000,0.080000,0.000000,0.030000,76.000000,29.000000,7.200000,96.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2014.000000,0.330000,0.290000,0.020000,0.100000,83.000000,44.000000,7.900000,388.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,2016.000000,7.020000,9.090000,1.870000,3.960000,98.000000,113.000000,9.300000,10665.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

Ubisoft Montreal          42
EA Sports                 36
Codemasters               30
EA Canada                 29
Visual Concepts           27
                          ..
EA Vancouver               1
Backbone Entertainment     1
WB Games Montreal          1
EA DICE, Danger Close      1
Coffee Stain Studios       1
Name: Developer, Length: 572, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(1363, 2114)

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100 , random_state=1)
model = brf.fit(X_train, y_train)
y_pred_brf = brf.predict(X_test)
brf

BalancedRandomForestClassifier(random_state=1)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_predictions = brf.predict(X_test)
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_predictions)

acc_score = balanced_accuracy_score(y_test, y_predictions)
print(acc_score)

0.6417370470200658


In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = brf.predict(X_test)
confusion_matrix(y_test, y_predictions)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_predictions))

                                                                   pre       rec       spe        f1       geo       iba       sup

                                           1C: Maddox Games       0.33      1.00      1.00      0.50      1.00      1.00         1
                                                  1C:Ino-Co       0.00      0.00      1.00      0.00      0.00      0.00         0
                                               2K Australia       1.00      1.00      1.00      1.00      1.00      1.00         1
                                                   2K Czech       1.00      1.00      1.00      1.00      1.00      1.00         1
                                                   2K Games       0.00      0.00      1.00      0.00      0.00      0.00         2
                                                  2K Sports       0.50      1.00      1.00      0.67      1.00      1.00         1
                                             343 Industries       0.50      1.00  

In [14]:
# List the features sorted in descending order by feature importance
features = sorted(zip(X.columns, brf.feature_importances_), key = lambda x: x[1], reverse=True)
cols = [brf for f in features]

features

[('Critic_Score', 0.02436493744664536),
 ('User_Count', 0.02411765360300105),
 ('User_Score', 0.023994459075944552),
 ('Critic_Count', 0.023483340839662775),
 ('EU_Sales', 0.023236129405621777),
 ('Year_of_Release', 0.021251771378300886),
 ('NA_Sales', 0.021029420204543556),
 ('Other_Sales', 0.019640418873804386),
 ('JP_Sales', 0.013660639070179538),
 ('Rating_T', 0.00936949040112741),
 ('Rating_M', 0.0074447966147663395),
 ('Genre_Action', 0.007060811545895277),
 ('Platform_PlayStation3', 0.006879172285425672),
 ('Platform_PC', 0.006211165439615065),
 ('Rating_E', 0.00609035106899352),
 ('Genre_Shooter', 0.0058712006821010024),
 ('Genre_Role-Playing', 0.005849203574087393),
 ('Rating_E10+', 0.005486765274639465),
 ('Publisher_Electronic Arts', 0.004904222573808244),
 ('Publisher_Ubisoft', 0.0048546747028944326),
 ('Platform_PlayStation4', 0.004779630948593864),
 ('Publisher_Sony Computer Entertainment', 0.0044034071163938406),
 ('Publisher_Activision', 0.0042663556474094085),
 ('Genre

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)
y_pred_eec = eec.predict(X_test)

In [16]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_predictions)

0.6417370470200658

In [17]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = eec.predict(X_test)
confusion_matrix(y_test, y_predictions)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                                                                                        pre       rec       spe        f1       geo       iba       sup

                                                                1C: Maddox Games       0.00      0.00      1.00      0.00      0.00      0.00         1
                                                                       1C:Ino-Co       0.00      0.00      0.97      0.00      0.00      0.00         0
                                                                    2K Australia       0.00      0.00      1.00      0.00      0.00      0.00         1
                                                                        2K Czech       0.00      0.00      1.00      0.00      0.00      0.00         1
                                                                        2K Games       0.00      0.00      1.00      0.00      0.00      0.00         2
                                                                       2K Sports       